In [58]:
import mysql.connector
from datetime import datetime
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox, simpledialog

def conectar():
    """Establece la conexión con la base de datos MySQL."""
    try:
        conn = mysql.connector.connect(
            host="shortline.proxy.rlwy.net",
            port=47316,
            user="root",
            password="cHHzHOJVpxFjPXLJsUQPHMNiZhCECXfJ",
            database="gamestore_db"
        )
        return conn
    except mysql.connector.Error as err:
        print(f"Error de conexión a la base de datos (desde conectar()): {err}")
        return None

In [59]:
class GameStoreManager:
    def __init__(self):
        self.db_conn = conectar()
        if self.db_conn:
            print("GameStoreManager: Conectado a la base de datos.")
            self._crear_tablas_si_no_existen()
        else:
            print("GameStoreManager: No se pudo conectar a la base de datos.")

    def _execute_query(self, query, params=None, fetch=None, commit=False, is_ddl=False):
        if not self.db_conn or not self.db_conn.is_connected():
            print("GameStoreManager: Intentando reconectar...")
            self.db_conn = conectar()
            if not self.db_conn:
                print("GameStoreManager: Fallo al reconectar.")
                return None if fetch else False

        use_dictionary_cursor = isinstance(fetch, str) and 'dict' in fetch
        cursor = None
        try:
            cursor = self.db_conn.cursor(dictionary=use_dictionary_cursor)
            # print(f"Executing query: {query[:200]} with params: {params}") # Debug line
            cursor.execute(query, params or ())
            if commit or is_ddl:
                self.db_conn.commit()
            if fetch:
                if fetch == 'one' or fetch == 'dict_one':
                    return cursor.fetchone()
                elif fetch == 'all' or fetch == 'dict_all':
                    return cursor.fetchall()
            return True if not fetch else cursor
        except mysql.connector.Error as err:
            print(f"GameStoreManager DB Error in _execute_query for query '{query[:100]}...': {err}")
            if self.db_conn:
                try:
                    # Si ocurre un error, y hay una transacción activa, intentar rollback.
                    if hasattr(self.db_conn, 'in_transaction') and self.db_conn.in_transaction:
                        print(f"GameStoreManager: _execute_query detectó transacción activa durante error. Intentando rollback.")
                        self.db_conn.rollback()
                        print(f"GameStoreManager: Rollback en _execute_query exitoso.")
                    # Adicionalmente, si la intención era commit/ddl, pero quizás la transacción no se marcó como activa
                    # o 'in_transaction' no está disponible, intentar rollback como antes.
                    elif (commit or is_ddl):
                        print(f"GameStoreManager: _execute_query error con intención de commit/ddl. Intentando rollback.")
                        self.db_conn.rollback()
                        print(f"GameStoreManager: Rollback (ruta commit/ddl) en _execute_query exitoso.")
                except mysql.connector.Error as roll_err:
                    print(f"GameStoreManager: Rollback Error en _execute_query: {roll_err}")
            return None if fetch else False
        finally:
            if cursor: cursor.close()

    def _crear_tablas_si_no_existen(self):
        queries = {
            "videojuegos": """
                CREATE TABLE IF NOT EXISTS videojuegos (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    nombre VARCHAR(255) NOT NULL,
                    precio DECIMAL(10, 2) NOT NULL,
                    consola VARCHAR(100),
                    genero VARCHAR(100),
                    stock INT NOT NULL DEFAULT 0
                );""",
            "clientes": """
                CREATE TABLE IF NOT EXISTS clientes (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    nombre VARCHAR(255) NOT NULL,
                    email VARCHAR(255) UNIQUE
                );""",
            "ventas": """
                CREATE TABLE IF NOT EXISTS ventas (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    id_cliente INT,
                    nombre_videojuego_vendido VARCHAR(255) NOT NULL,
                    precio_venta DECIMAL(10, 2) NOT NULL,
                    fecha_venta TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    FOREIGN KEY (id_cliente) REFERENCES clientes(id) ON DELETE SET NULL
                );"""
        }
        for tabla, query in queries.items():
            cursor_check = self.db_conn.cursor()
            cursor_check.execute(f"SHOW TABLES LIKE '{tabla}'")
            result = cursor_check.fetchone()
            cursor_check.close()

            if not result:
                 if self._execute_query(query, is_ddl=True):
                    print(f"GameStoreManager: Tabla '{tabla}' creada.")
                 else:
                    print(f"GameStoreManager: Error al crear tabla '{tabla}'.")
            else: 
                # print(f"GameStoreManager: Tabla '{tabla}' ya existe.") # Reduced verbosity
                if tabla == "videojuegos":
                    cursor_col_check = self.db_conn.cursor(dictionary=True)
                    cursor_col_check.execute(f"SHOW COLUMNS FROM videojuegos LIKE 'stock';")
                    stock_col_exists = cursor_col_check.fetchone()
                    cursor_col_check.close()
                    if not stock_col_exists:
                        print(f"GameStoreManager: Columna 'stock' no encontrada en tabla '{tabla}'. Intentando agregarla...")
                        alter_query = "ALTER TABLE videojuegos ADD COLUMN stock INT NOT NULL DEFAULT 0"
                        if self._execute_query(alter_query, is_ddl=True):
                            print(f"GameStoreManager: Columna 'stock' agregada a tabla '{tabla}'.")
                        else:
                            print(f"GameStoreManager: Error al agregar columna 'stock' a tabla '{tabla}'. ¡Por favor, verifica manualmente o borra la tabla '{tabla}' y reintenta!")
                    
                    cursor_col_check_cat = self.db_conn.cursor(dictionary=True)
                    cursor_col_check_cat.execute(f"SHOW COLUMNS FROM videojuegos LIKE 'categoria';")
                    cat_col_exists = cursor_col_check_cat.fetchone()
                    cursor_col_check_cat.close()
                    if cat_col_exists:
                        print(f"GameStoreManager: Columna obsoleta 'categoria' encontrada en tabla '{tabla}'. Intentando eliminarla...")
                        alter_query_drop_cat = "ALTER TABLE videojuegos DROP COLUMN categoria"
                        if self._execute_query(alter_query_drop_cat, is_ddl=True):
                            print(f"GameStoreManager: Columna 'categoria' eliminada de tabla '{tabla}'.")
                        else:
                            print(f"GameStoreManager: Error al eliminar columna 'categoria' de tabla '{tabla}'.")

    def is_connected(self):
        return self.db_conn and self.db_conn.is_connected()

    def add_videojuego_gui(self, nombre, precio, consola, genero, stock):
        # ... (sin cambios)
        if not nombre or precio is None or stock is None:
            return False, "Nombre, Precio y Stock son obligatorios."
        try:
            precio_float = float(precio)
            if precio_float < 0:
                return False, "El precio no puede ser negativo."
        except ValueError:
            return False, "El precio debe ser un número válido."
        try:
            stock_int = int(stock)
            if stock_int < 0:
                return False, "El stock no puede ser negativo."
        except ValueError:
            return False, "El stock debe ser un número entero válido."

        query = "INSERT INTO videojuegos (nombre, precio, consola, genero, stock) VALUES (%s, %s, %s, %s, %s)"
        params = (nombre, precio_float, consola, genero, stock_int)
        if self._execute_query(query, params, commit=True):
            return True, f"Videojuego '{nombre}' agregado exitosamente con stock {stock_int}."
        else:
            return False, f"Error al agregar videojuego '{nombre}'."

    def add_cliente_gui(self, nombre, email):
        # ... (sin cambios)
        if not nombre:
            return False, "El nombre del cliente es obligatorio."
        email_param = email if email else None
        query = "INSERT INTO clientes (nombre, email) VALUES (%s, %s)"
        params = (nombre, email_param)
        if self._execute_query(query, params, commit=True):
            return True, f"Cliente '{nombre}' agregado exitosamente."
        else:
            return False, f"Error al agregar cliente '{nombre}'. Verifique si el email ya existe."

    def delete_cliente_gui(self, cliente_id):
        # ... (sin cambios)
        if cliente_id is None:
            return False, "No se ha seleccionado un cliente."
        query = "DELETE FROM clientes WHERE id = %s"
        if self._execute_query(query, (cliente_id,), commit=True):
            return True, f"Cliente ID {cliente_id} eliminado exitosamente."
        else:
            return False, f"Error al eliminar el cliente ID {cliente_id}."

    def get_catalogo_videojuegos(self):
        # ... (sin cambios)
        query = "SELECT id, nombre, precio, consola, genero, stock FROM videojuegos ORDER BY nombre"
        return self._execute_query(query, fetch='dict_all')

    def get_clientes_for_selection(self):
        # ... (sin cambios)
        query = "SELECT id, nombre, email FROM clientes ORDER BY nombre"
        return self._execute_query(query, fetch='dict_all')

    def get_videojuego_by_id(self, juego_id):
        # ... (sin cambios)
        query = "SELECT id, nombre, precio, stock, consola, genero FROM videojuegos WHERE id = %s"
        return self._execute_query(query, (juego_id,), fetch='dict_one')

    def realizar_venta_gui(self, id_videojuego, nombre_videojuego_vendido, precio_venta, id_cliente):
        cliente_id_param = id_cliente if id_cliente is not None else None

        if not self.db_conn or not self.db_conn.is_connected():
            print("GameStoreManager: Realizar Venta - Intentando reconectar...")
            self.db_conn = conectar()
            if not self.db_conn:
                return False, "Fallo al reconectar para la venta."
        
        cursor = None
        try:
            if hasattr(self.db_conn, 'in_transaction') and self.db_conn.in_transaction:
                print("GameStoreManager: Advertencia (realizar_venta_gui) - La conexión ya estaba en una transacción. Intentando rollback pre-emptivo.")
                try:
                    self.db_conn.rollback()
                    print("GameStoreManager: Rollback pre-emptivo en realizar_venta_gui exitoso.")
                except mysql.connector.Error as pre_roll_err:
                    print(f"GameStoreManager: Error en rollback pre-emptivo (realizar_venta_gui): {pre_roll_err}.")
                    # Considerar si es seguro continuar o si se debe retornar un error.
                    # return False, f"Error crítico al limpiar transacción previa: {pre_roll_err}"

            game_details = self.get_videojuego_by_id(id_videojuego)
            if not game_details:
                return False, f"Videojuego '{nombre_videojuego_vendido}' (ID: {id_videojuego}) no encontrado."
            if game_details['stock'] <= 0:
                return False, f"No hay stock disponible para '{nombre_videojuego_vendido}'."

            cursor = self.db_conn.cursor()
            
            if hasattr(self.db_conn, 'start_transaction'):
                 self.db_conn.start_transaction()
            else: 
                 cursor.execute("START TRANSACTION")

            query_venta = "INSERT INTO ventas (id_cliente, nombre_videojuego_vendido, precio_venta, fecha_venta) VALUES (%s, %s, %s, %s)"
            fecha_actual = datetime.now()
            params_venta = (cliente_id_param, nombre_videojuego_vendido, precio_venta, fecha_actual)
            cursor.execute(query_venta, params_venta)

            query_update_stock = "UPDATE videojuegos SET stock = stock - 1 WHERE id = %s"
            cursor.execute(query_update_stock, (id_videojuego,))
            
            self.db_conn.commit()
            return True, f"Venta de '{nombre_videojuego_vendido}' completada. Stock actualizado."

        except mysql.connector.Error as err: 
            print(f"GameStoreManager DB Error during sale processing (realizar_venta_gui): {err}") 
            if self.db_conn:
                try:
                    # Este rollback es para la transacción iniciada en *este* intento de realizar_venta_gui
                    if hasattr(self.db_conn, 'in_transaction') and self.db_conn.in_transaction:
                        self.db_conn.rollback() 
                        print("GameStoreManager: Transacción de venta (o intento) revertida debido a error en realizar_venta_gui.")
                    else:
                        print("GameStoreManager: No se detectó transacción activa para revertir en el except de realizar_venta_gui, pero hubo un error.")
                except mysql.connector.Error as roll_err:
                    print(f"GameStoreManager: Error al intentar revertir transacción en except block (realizar_venta_gui): {roll_err}.")
            return False, f"Error durante la transacción de venta: {err}"
        finally:
            if cursor:
                cursor.close()

    def get_all_sales_report(self):
        # ... (sin cambios)
        query = """
            SELECT v.id, c.nombre AS nombre_cliente, v.nombre_videojuego_vendido, v.precio_venta, v.fecha_venta
            FROM ventas v
            LEFT JOIN clientes c ON v.id_cliente = c.id
            ORDER BY v.fecha_venta DESC;
        """
        return self._execute_query(query, fetch='dict_all')

    def get_total_revenue_report(self):
        # ... (sin cambios)
        query = "SELECT SUM(precio_venta) AS total FROM ventas"
        resultado = self._execute_query(query, fetch='dict_one')
        if resultado and resultado['total'] is not None:
            return resultado['total'], "Ingresos totales calculados."
        elif resultado and resultado['total'] is None:
            return 0.0, "No hay ventas registradas."
        else:
            return None, "Error al calcular ingresos totales."

    def get_sales_by_client_report(self):
        # ... (sin cambios)
        query = """
            SELECT c.nombre AS nombre_cliente, COUNT(v.id) AS numero_ventas, SUM(v.precio_venta) AS total_gastado
            FROM clientes c
            JOIN ventas v ON c.id = v.id_cliente
            GROUP BY c.id, c.nombre
            ORDER BY total_gastado DESC;
        """
        report_data = self._execute_query(query, fetch='dict_all')
        if report_data is None:
            return None, "Error al generar el informe de ventas por cliente."
        if not report_data:
            return [], "No hay ventas registradas o clientes con ventas."
        return report_data, "Informe de ventas por cliente generado."

    def cerrar_conexion(self):
        # ... (sin cambios, el rollback pre-emptivo ya estaba aquí)
        if self.db_conn and self.db_conn.is_connected():
            if hasattr(self.db_conn, 'in_transaction') and self.db_conn.in_transaction:
                print("GameStoreManager: Advertencia - Cerrando conexión con transacción activa. Intentando rollback.")
                try:
                    self.db_conn.rollback()
                except mysql.connector.Error as e:
                    print(f"GameStoreManager: Error en rollback durante cierre de conexión: {e}")
            self.db_conn.close()
            print("GameStoreManager: Conexión cerrada.")

In [60]:
class GameStoreApp:
    def __init__(self, root_window):
        self.root = root_window
        self.root.title("GameStore Manager")
        self.root.geometry("850x650")

        self.manager = GameStoreManager() # Esto ejecutará _crear_tablas_si_no_existen

        if not self.manager.is_connected():
            messagebox.showerror("Error de Conexión",
                                 "No se pudo conectar a la base de datos.\n"
                                 "La aplicación no funcionará correctamente.\n"
                                 "Revise la consola para más detalles.")
            # Consider disabling UI elements or exiting if connection fails critically
        
        style = ttk.Style()
        style.theme_use('clam') 

        menu_frame = ttk.Frame(self.root, padding="10")
        menu_frame.pack(side=tk.TOP, fill=tk.X)

        ttk.Button(menu_frame, text="Agregar Videojuego", command=self.open_add_videojuego_window).pack(side=tk.LEFT, padx=5)
        ttk.Button(menu_frame, text="Agregar Cliente", command=self.open_add_cliente_window).pack(side=tk.LEFT, padx=5)
        ttk.Button(menu_frame, text="Borrar Cliente", command=self.open_delete_cliente_window).pack(side=tk.LEFT, padx=5)
        ttk.Button(menu_frame, text="Realizar Venta", command=self.open_realizar_venta_window).pack(side=tk.LEFT, padx=5)
        ttk.Button(menu_frame, text="Informes de Ventas", command=self.open_informes_window).pack(side=tk.LEFT, padx=5)

        self.content_frame = ttk.Frame(self.root, padding="10")
        self.content_frame.pack(expand=True, fill=tk.BOTH)
        
        # Only show catalog if manager is connected and tables likely exist
        if self.manager.is_connected():
            self.show_catalogo_in_content_frame()
        else:
             ttk.Label(self.content_frame, text="No se puede mostrar el catálogo. Verifique la conexión a la BD.", foreground="red").pack(pady=20)


    def clear_content_frame(self):
        for widget in self.content_frame.winfo_children():
            widget.destroy()

    def open_add_videojuego_window(self):
        if not self.manager.is_connected():
            messagebox.showerror("Error", "No hay conexión a la base de datos.")
            return

        top = tk.Toplevel(self.root)
        top.title("Agregar Videojuego")
        top.geometry("400x300")

        fields = ["Nombre", "Precio", "Consola", "Género", "Stock"]
        entries = {}

        form_frame = ttk.Frame(top, padding="10")
        form_frame.pack(expand=True, fill=tk.BOTH)

        for i, field in enumerate(fields):
            ttk.Label(form_frame, text=field + ":").grid(row=i, column=0, padx=5, pady=5, sticky=tk.W)
            entry = ttk.Entry(form_frame, width=30)
            entry.grid(row=i, column=1, padx=5, pady=5, sticky=tk.EW)
            entries[field] = entry
            if field == "Stock":
                entry.insert(0, "0")
        
        form_frame.grid_columnconfigure(1, weight=1)

        def submit_videojuego():
            data = {field: entries[field].get().strip() for field in fields}
            if not data["Nombre"] or not data["Precio"] or not data["Stock"]:
                messagebox.showerror("Error de Validación", "Nombre, Precio y Stock son obligatorios.", parent=top)
                return
            try:
                float(data["Precio"])
            except ValueError:
                messagebox.showerror("Error de Validación", "El precio debe ser un número.", parent=top)
                return
            try:
                stock_val = int(data["Stock"])
                if stock_val < 0:
                    messagebox.showerror("Error de Validación", "El stock no puede ser negativo.", parent=top)
                    return
            except ValueError:
                messagebox.showerror("Error de Validación", "El stock debe ser un número entero.", parent=top)
                return

            success, message = self.manager.add_videojuego_gui(
                data["Nombre"], data["Precio"], data["Consola"], data["Género"], data["Stock"]
            )
            if success:
                messagebox.showinfo("Éxito", message, parent=top)
                top.destroy()
                self.show_catalogo_in_content_frame() 
            else:
                messagebox.showerror("Error", message, parent=top)

        ttk.Button(form_frame, text="Agregar", command=submit_videojuego).grid(row=len(fields), column=0, columnspan=2, pady=10)
        top.transient(self.root) 
        top.grab_set() 
        self.root.wait_window(top) 

    def open_add_cliente_window(self):
        if not self.manager.is_connected():
            messagebox.showerror("Error", "No hay conexión a la base de datos.")
            return
            
        top = tk.Toplevel(self.root)
        top.title("Agregar Cliente")
        top.geometry("350x200")

        fields = ["Nombre", "Email"]
        entries = {}
        form_frame = ttk.Frame(top, padding="10")
        form_frame.pack(expand=True, fill=tk.BOTH)

        for i, field in enumerate(fields):
            ttk.Label(form_frame, text=field + ":").grid(row=i, column=0, padx=5, pady=5, sticky=tk.W)
            entry = ttk.Entry(form_frame, width=30)
            entry.grid(row=i, column=1, padx=5, pady=5, sticky=tk.EW)
            entries[field] = entry
        
        form_frame.grid_columnconfigure(1, weight=1)

        def submit_cliente():
            data = {field: entries[field].get().strip() for field in fields}
            if not data["Nombre"]:
                messagebox.showerror("Error de Validación", "El nombre es obligatorio.", parent=top)
                return

            success, message = self.manager.add_cliente_gui(data["Nombre"], data["Email"])
            if success:
                messagebox.showinfo("Éxito", message, parent=top)
                top.destroy()
            else:
                messagebox.showerror("Error", message, parent=top)
        
        ttk.Button(form_frame, text="Agregar", command=submit_cliente).grid(row=len(fields), column=0, columnspan=2, pady=10)
        top.transient(self.root)
        top.grab_set()
        self.root.wait_window(top)

    def open_delete_cliente_window(self):
        if not self.manager.is_connected():
            messagebox.showerror("Error", "No hay conexión a la base de datos.")
            return

        top = tk.Toplevel(self.root)
        top.title("Borrar Cliente")
        top.geometry("450x250")

        selection_frame = ttk.Frame(top, padding="10")
        selection_frame.pack(fill=tk.X, expand=True)
        
        action_frame = ttk.Frame(top, padding="10")
        action_frame.pack(fill=tk.X)

        ttk.Label(selection_frame, text="Seleccionar Cliente:").grid(row=0, column=0, padx=5, pady=5, sticky=tk.W)
        
        clientes = self.manager.get_clientes_for_selection()
        cliente_options = []
        self.delete_cliente_map = {}
        if clientes:
            for c in clientes:
                display_name = f"{c['nombre']} (ID: {c['id']}, Email: {c.get('email', 'N/A')})"
                cliente_options.append(display_name)
                self.delete_cliente_map[display_name] = c['id']
        
        self.selected_cliente_to_delete_str = tk.StringVar()
        cliente_combo = ttk.Combobox(selection_frame, textvariable=self.selected_cliente_to_delete_str, 
                                     values=cliente_options, width=50, state="readonly")
        cliente_combo.grid(row=0, column=1, padx=5, pady=5, sticky=tk.EW)
        if not cliente_options:
            cliente_combo['values'] = ["No hay clientes para borrar"]
            cliente_combo.current(0)
            cliente_combo.config(state="disabled")
        elif cliente_options:
            cliente_combo.current(0)
        
        selection_frame.grid_columnconfigure(1, weight=1)

        def submit_delete_cliente():
            cliente_display_name = self.selected_cliente_to_delete_str.get()
            if not cliente_display_name or "No hay clientes" in cliente_display_name:
                messagebox.showerror("Error", "Debe seleccionar un cliente para borrar.", parent=top)
                return

            cliente_id_to_delete = self.delete_cliente_map.get(cliente_display_name)
            if cliente_id_to_delete is None:
                messagebox.showerror("Error", "Selección de cliente inválida.", parent=top)
                return
            
            confirm = messagebox.askyesno("Confirmar Borrado", 
                                          f"¿Seguro que quieres borrar al cliente '{cliente_display_name}'?\n"
                                          "Las ventas asociadas no se eliminarán, pero perderán la referencia directa al cliente.", 
                                          parent=top)
            if not confirm:
                return

            success, message = self.manager.delete_cliente_gui(cliente_id_to_delete)
            if success:
                messagebox.showinfo("Éxito", message, parent=top)
                top.destroy()
            else:
                messagebox.showerror("Error al Borrar", message, parent=top)

        ttk.Button(action_frame, text="Borrar Cliente Seleccionado", command=submit_delete_cliente).pack(pady=10)
        
        top.transient(self.root)
        top.grab_set()
        self.root.wait_window(top)

    def show_catalogo_in_content_frame(self, event=None): 
        self.clear_content_frame()
        if not self.manager.is_connected():
            ttk.Label(self.content_frame, text="No hay conexión a la base de datos para mostrar el catálogo.", foreground="red").pack(pady=20)
            return

        catalogo = self.manager.get_catalogo_videojuegos()

        if catalogo is None: 
            ttk.Label(self.content_frame, text="Error al cargar el catálogo. Verifique la consola.", foreground="red").pack(pady=20)
            return
        if not catalogo: 
            ttk.Label(self.content_frame, text="El catálogo de videojuegos está vacío.").pack(pady=20)
            return

        cols = ('ID', 'Nombre', 'Precio', 'Consola', 'Género', 'Stock')
        tree = ttk.Treeview(self.content_frame, columns=cols, show='headings', selectmode="browse")

        for col in cols:
            tree.heading(col, text=col)
            tree.column(col, width=100, anchor=tk.W) 
        tree.column('Nombre', width=180)
        tree.column('Precio', anchor=tk.E, width=80)
        tree.column('Stock', anchor=tk.CENTER, width=60)

        for juego in catalogo:
            tree.insert("", tk.END, values=(
                juego['id'], juego['nombre'], f"${juego['precio']:.2f}", 
                juego.get('consola', ''), juego.get('genero', ''), juego.get('stock', 0)
            ))
        
        vsb = ttk.Scrollbar(self.content_frame, orient="vertical", command=tree.yview)
        hsb = ttk.Scrollbar(self.content_frame, orient="horizontal", command=tree.xview)
        tree.configure(yscrollcommand=vsb.set, xscrollcommand=hsb.set)

        tree.pack(side=tk.LEFT, expand=True, fill=tk.BOTH)
        vsb.pack(side=tk.RIGHT, fill=tk.Y)
        hsb.pack(side=tk.BOTTOM, fill=tk.X)

    def open_realizar_venta_window(self):
        if not self.manager.is_connected():
            messagebox.showerror("Error", "No hay conexión a la base de datos.")
            return

        top = tk.Toplevel(self.root)
        top.title("Realizar Venta")
        top.geometry("650x300")

        selection_frame = ttk.Frame(top, padding="10")
        selection_frame.pack(fill=tk.X)
        
        action_frame = ttk.Frame(top, padding="10")
        action_frame.pack(fill=tk.X)

        ttk.Label(selection_frame, text="Videojuego:").grid(row=0, column=0, padx=5, pady=5, sticky=tk.W)
        juegos_disponibles = self.manager.get_catalogo_videojuegos()
        juego_options = []
        self.juego_map = {} 
        if juegos_disponibles:
            for j in juegos_disponibles:
                if j['stock'] > 0:
                    display_name = f"{j['nombre']} (${j['precio']:.2f}) - Stock: {j['stock']}"
                    juego_options.append(display_name)
                    self.juego_map[display_name] = {'id': j['id'], 'nombre': j['nombre'], 'precio': j['precio'], 'stock': j['stock']}
        
        self.selected_juego_str = tk.StringVar()
        juego_combo = ttk.Combobox(selection_frame, textvariable=self.selected_juego_str, values=juego_options, width=60, state="readonly")
        juego_combo.grid(row=0, column=1, padx=5, pady=5, sticky=tk.EW)
        if not juego_options:
            juego_combo['values'] = ["No hay juegos con stock disponibles"]
            juego_combo.current(0)
            juego_combo.config(state="disabled")
        elif juego_options: # Ensure current(0) is only called if options exist
            juego_combo.current(0)

        ttk.Label(selection_frame, text="Cliente:").grid(row=1, column=0, padx=5, pady=5, sticky=tk.W)
        clientes_disponibles = self.manager.get_clientes_for_selection()
        cliente_options = []
        self.cliente_map = {} 
        if clientes_disponibles:
            for c in clientes_disponibles:
                display_name = f"{c['nombre']} (ID: {c['id']})" 
                cliente_options.append(display_name)
                self.cliente_map[display_name] = c['id']
        
        self.selected_cliente_str = tk.StringVar()
        cliente_combo = ttk.Combobox(selection_frame, textvariable=self.selected_cliente_str, values=cliente_options, width=60, state="readonly")
        cliente_combo.grid(row=1, column=1, padx=5, pady=5, sticky=tk.EW)
        if not cliente_options:
            cliente_combo['values'] = ["No hay clientes registrados"]
            cliente_combo.current(0)
            cliente_combo.config(state="disabled")
        elif cliente_options: # Ensure current(0) is only called if options exist
            cliente_combo.current(0)
        
        selection_frame.grid_columnconfigure(1, weight=1) 

        def submit_venta():
            juego_display_name = self.selected_juego_str.get()
            cliente_display_name = self.selected_cliente_str.get()

            if not juego_display_name or "No hay juegos" in juego_display_name:
                messagebox.showerror("Error", "Debe seleccionar un videojuego con stock disponible.", parent=top)
                return
            if not cliente_display_name or "No hay clientes" in cliente_display_name:
                 messagebox.showerror("Error", "Debe seleccionar un cliente.", parent=top)
                 return

            juego_seleccionado = self.juego_map.get(juego_display_name)
            id_cliente_seleccionado = self.cliente_map.get(cliente_display_name)

            if not juego_seleccionado:
                messagebox.showerror("Error", "Videojuego seleccionado no válido o sin stock.", parent=top)
                return # Refreshing logic moved to after failed sale below for clarity
            if id_cliente_seleccionado is None: 
                messagebox.showerror("Error", "Cliente seleccionado no válido.", parent=top)
                return
            
            # Re-check stock before confirmation, as it might have changed
            current_game_state = self.manager.get_videojuego_by_id(juego_seleccionado['id'])
            if not current_game_state or current_game_state['stock'] <= 0:
                messagebox.showerror("Error", f"'{juego_seleccionado['nombre']}' ya no tiene stock.", parent=top)
                # Refresh game list in combobox
                self.refresh_juego_combo_options_for_sale(juego_combo)
                return

            confirm = messagebox.askyesno("Confirmar Venta", 
                                          f"Vender '{juego_seleccionado['nombre']}' por ${juego_seleccionado['precio']:.2f}\n"
                                          f"al cliente '{cliente_display_name}'?", 
                                          parent=top)
            if not confirm:
                return

            success, message = self.manager.realizar_venta_gui(
                juego_seleccionado['id'], 
                juego_seleccionado['nombre'], 
                juego_seleccionado['precio'], 
                id_cliente_seleccionado
            )

            if success:
                messagebox.showinfo("Éxito", message, parent=top)
                top.destroy()
                self.show_catalogo_in_content_frame() 
            else:
                messagebox.showerror("Error en Venta", message, parent=top)
                if "stock" in message.lower(): # If error was stock related, refresh
                    self.refresh_juego_combo_options_for_sale(juego_combo)

        ttk.Button(action_frame, text="Confirmar Venta", command=submit_venta).pack(pady=10)
        
        top.transient(self.root)
        top.grab_set()
        self.root.wait_window(top)

    def refresh_juego_combo_options_for_sale(self, juego_combo_widget):
        """Helper to refresh game options in the sale window's combobox."""
        juegos_act = self.manager.get_catalogo_videojuegos()
        updated_juego_options = []
        new_juego_map = {}
        if juegos_act:
            for j_act in juegos_act:
                if j_act['stock'] > 0:
                    d_name = f"{j_act['nombre']} (${j_act['precio']:.2f}) - Stock: {j_act['stock']}"
                    updated_juego_options.append(d_name)
                    new_juego_map[d_name] = {'id': j_act['id'], 'nombre': j_act['nombre'], 'precio': j_act['precio'], 'stock': j_act['stock']}
        
        current_selection = juego_combo_widget.get()
        juego_combo_widget['values'] = updated_juego_options if updated_juego_options else ["No hay juegos con stock disponibles"]
        self.juego_map = new_juego_map # Update the app's map

        if updated_juego_options:
            if current_selection in updated_juego_options: # try to reselect if still valid
                 juego_combo_widget.set(current_selection)
            else:
                 juego_combo_widget.current(0)
            juego_combo_widget.config(state="readonly")
        else:
            juego_combo_widget.current(0) # Select "No hay juegos..."
            juego_combo_widget.config(state="disabled")


    def open_informes_window(self):
        if not self.manager.is_connected():
            messagebox.showerror("Error", "No hay conexión a la base de datos.")
            return

        top = tk.Toplevel(self.root)
        top.title("Informes de Ventas")
        top.geometry("750x550")

        informe_buttons_frame = ttk.Frame(top, padding="10")
        informe_buttons_frame.pack(fill=tk.X)

        informe_display_frame = ttk.Frame(top, padding="10")
        informe_display_frame.pack(expand=True, fill=tk.BOTH)
        
        self.report_container = ttk.Frame(informe_display_frame) 
        self.report_container.pack(expand=True, fill=tk.BOTH)

        def show_all_sales_report():
            self.clear_report_container()
            sales_data = self.manager.get_all_sales_report()
            if sales_data is None: 
                ttk.Label(self.report_container, text="Error al obtener el informe de todas las ventas.", foreground="red").pack()
                return
            if not sales_data: 
                ttk.Label(self.report_container, text="No hay ventas registradas.").pack()
                return

            cols = ('ID Venta', 'Cliente', 'Videojuego Vendido', 'Precio Venta', 'Fecha')
            tree = ttk.Treeview(self.report_container, columns=cols, show='headings')
            for col in cols:
                tree.heading(col, text=col)
                tree.column(col, width=130, anchor=tk.W)
            tree.column('Precio Venta', anchor=tk.E, width=100)
            tree.column('Fecha', width=160)

            for venta in sales_data:
                tree.insert("", tk.END, values=(
                    venta['id'],
                    venta.get('nombre_cliente', 'N/A (Cliente Borrado)'), 
                    venta['nombre_videojuego_vendido'],
                    f"${venta['precio_venta']:.2f}",
                    venta['fecha_venta'].strftime('%Y-%m-%d %H:%M:%S') if venta.get('fecha_venta') else 'N/A'
                ))
            
            vsb = ttk.Scrollbar(self.report_container, orient="vertical", command=tree.yview)
            hsb = ttk.Scrollbar(self.report_container, orient="horizontal", command=tree.xview)
            tree.configure(yscrollcommand=vsb.set, xscrollcommand=hsb.set)
            
            tree.pack(side=tk.LEFT, expand=True, fill=tk.BOTH)
            vsb.pack(side=tk.RIGHT, fill=tk.Y)
            hsb.pack(side=tk.BOTTOM, fill=tk.X)

        def show_total_revenue():
            self.clear_report_container()
            total, message = self.manager.get_total_revenue_report()
            if total is not None:
                ttk.Label(self.report_container, text=f"Ingresos Totales: ${total:.2f}", font=("Arial", 16, "bold")).pack(pady=30)
            else: 
                ttk.Label(self.report_container, text=message, foreground="red", font=("Arial", 14)).pack(pady=20)

        def show_sales_by_client():
            self.clear_report_container()
            data, message = self.manager.get_sales_by_client_report()

            if data is None: 
                ttk.Label(self.report_container, text=message, foreground="red").pack()
                return
            if not data: 
                ttk.Label(self.report_container, text="No hay ventas agrupadas por cliente para mostrar.").pack()
                return

            cols = ('Cliente', 'Nº Ventas', 'Total Gastado')
            tree = ttk.Treeview(self.report_container, columns=cols, show='headings')
            for col in cols:
                tree.heading(col, text=col)
                tree.column(col, width=180, anchor=tk.W)
            tree.column('Total Gastado', anchor=tk.E, width=120)
            tree.column('Nº Ventas', anchor=tk.CENTER, width=100)

            for row in data:
                tree.insert("", tk.END, values=(
                    row['nombre_cliente'],
                    row['numero_ventas'],
                    f"${row['total_gastado']:.2f}"
                ))
            
            vsb = ttk.Scrollbar(self.report_container, orient="vertical", command=tree.yview)
            hsb = ttk.Scrollbar(self.report_container, orient="horizontal", command=tree.xview)
            tree.configure(yscrollcommand=vsb.set, xscrollcommand=hsb.set)

            tree.pack(side=tk.LEFT, expand=True, fill=tk.BOTH)
            vsb.pack(side=tk.RIGHT, fill=tk.Y)
            hsb.pack(side=tk.BOTTOM, fill=tk.X)

        ttk.Button(informe_buttons_frame, text="Todas las Ventas", command=show_all_sales_report).pack(side=tk.LEFT, padx=5)
        ttk.Button(informe_buttons_frame, text="Ingresos Totales", command=show_total_revenue).pack(side=tk.LEFT, padx=5)
        ttk.Button(informe_buttons_frame, text="Ventas por Cliente", command=show_sales_by_client).pack(side=tk.LEFT, padx=5)
        
        top.transient(self.root)
        top.grab_set()
        self.root.wait_window(top)

    def clear_report_container(self):
        for widget in self.report_container.winfo_children():
            widget.destroy()

In [ ]:
# Asegúrate de haber ejecutado las celdas anteriores que definen:
# - conectar()
# - GameStoreManager
# - GameStoreApp

# Crear y lanzar la aplicación
try:
    # Intenta cerrar cualquier instancia de Tkinter previa si estás re-ejecutando en Jupyter
    if 'root' in globals() and root.winfo_exists():
        print("Cerrando ventana Tkinter previa...")
        if app and app.manager:
            app.manager.cerrar_conexion()
        root.destroy()
except NameError: # 'root' or 'app' might not be defined on first run
    pass
except tk.TclError: # If root was already destroyed
    pass


print("Lanzando GameStoreApp...")
root = tk.Tk()
app = GameStoreApp(root)

def on_closing_jupyter():
    if messagebox.askokcancel("Salir", "¿Seguro que quieres salir de GameStore?"):
        print("Cerrando conexión a la base de datos...")
        if app.manager: 
            app.manager.cerrar_conexion()
        print("Destruyendo ventana principal...")
        root.destroy()
        print("Aplicación cerrada.")

root.protocol("WM_DELETE_WINDOW", on_closing_jupyter) 
root.mainloop()
print("mainloop de Tkinter finalizado.")

Lanzando GameStoreApp...
GameStoreManager: Conectado a la base de datos.
GameStoreManager: Advertencia (realizar_venta_gui) - La conexión ya estaba en una transacción. Intentando rollback pre-emptivo.
GameStoreManager: Rollback pre-emptivo en realizar_venta_gui exitoso.
GameStoreManager DB Error during sale processing (realizar_venta_gui): Transaction already in progress
GameStoreManager: Transacción de venta (o intento) revertida debido a error en realizar_venta_gui.


In [ ]:
if __name__ == "__main__":
    # Esta condición __name__ == "__main__" no se cumple directamente en celdas de Jupyter
    # al importar o ejecutar toda la celda.
    # Para ejecutar la app Tkinter en Jupyter, usualmente se llama directamente
    # la creación de la ventana y el mainloop.
    # Sin embargo, si guardas este script como .py y lo ejecutas, funcionará como antes.

    # Para correr en Jupyter, puedes hacer esto en una celda separada
    # DESPUÉS de ejecutar las celdas con las definiciones de clase:
    
    # root = tk.Tk()
    # app = GameStoreApp(root)
    #
    # def on_closing():
    # if messagebox.askokcancel("Salir", "¿Seguro que quieres salir?"):
    # if app.manager:
    # app.manager.cerrar_conexion()
    #         root.destroy()
    #
    # root.protocol("WM_DELETE_WINDOW", on_closing)
    # root.mainloop()
    
    # La forma estándar de ejecutarlo como script es:
    pass # Dejar pass aquí si solo defines las clases en esta celda
         # y ejecutas la app en la siguiente celda.

In [ ]:
# Asegúrate de haber ejecutado las celdas anteriores que definen:
# - conectar()
# - GameStoreManager
# - GameStoreApp

# Crear y lanzar la aplicación
try:
    if 'root' in globals() and root.winfo_exists():
        print("Cerrando ventana Tkinter previa...")
        if 'app' in globals() and app and app.manager:
            app.manager.cerrar_conexion()
        root.destroy()
except NameError: 
    pass
except tk.TclError: 
    pass
except Exception as e:
    print(f"Error al intentar cerrar ventana previa: {e}")


print("Lanzando GameStoreApp...")
root = tk.Tk()
app = GameStoreApp(root)

closing_in_progress = False # Bandera para evitar re-entrada

def on_closing_jupyter():
    global closing_in_progress
    if closing_in_progress:
        print("on_closing_jupyter: Cierre ya en progreso, ignorando.")
        return
    
    closing_in_progress = True
    print("WM_DELETE_WINDOW triggered (Protocolo de Cierre)")

    try:
        if messagebox.askokcancel("Salir", "¿Seguro que quieres salir de GameStore?"):
            print("Usuario confirmó salir. Procediendo con el cierre...")
            if app.manager:
                print("Llamando a app.manager.cerrar_conexion()...")
                try:
                    app.manager.cerrar_conexion()
                    print("app.manager.cerrar_conexion() finalizado.")
                except Exception as e_conn:
                    print(f"ERROR durante app.manager.cerrar_conexion(): {e_conn}")
            
            print("Llamando a root.destroy()...")
            try:
                root.destroy()
                print("root.destroy() finalizado. La aplicación debería cerrarse.")
            except Exception as e_destroy:
                print(f"ERROR durante root.destroy(): {e_destroy}")
            print("Aplicación (lógica de cierre) finalizada.")
        else:
            print("Usuario canceló el cierre.")
    except Exception as e_closing:
        print(f"ERROR inesperado en on_closing_jupyter: {e_closing}")
    finally:
        closing_in_progress = False # Restablecer la bandera

root.protocol("WM_DELETE_WINDOW", on_closing_jupyter) 
print("Iniciando root.mainloop()...")
root.mainloop()
print("mainloop de Tkinter finalizado.")

Lanzando GameStoreApp...
GameStoreManager: Conectado a la base de datos.
Iniciando root.mainloop()...
WM_DELETE_WINDOW triggered (Protocolo de Cierre)
Usuario confirmó salir. Procediendo con el cierre...
Llamando a app.manager.cerrar_conexion()...
GameStoreManager: Advertencia - Cerrando conexión con transacción activa. Intentando rollback.
GameStoreManager: Conexión cerrada.
app.manager.cerrar_conexion() finalizado.
Llamando a root.destroy()...
root.destroy() finalizado. La aplicación debería cerrarse.
Aplicación (lógica de cierre) finalizada.
mainloop de Tkinter finalizado.
